In [95]:
%load_ext autoreload
%autoreload 2

import json
from gridfs import GridFS, NoFile
from flask import Flask
from flask_cors import CORS, cross_origin
from pymongo import MongoClient
from bson.objectid import ObjectId
from db_interface import addMagresFile, databaseSearch, removeMagresFiles

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# MAGRES database
client = MongoClient(host='wigner.esc.rl.ac.uk')
ccpnc = client.ccpnc

In [35]:
magresFilesFS = GridFS(ccpnc, 'magresFilesFS')

In [84]:
new_id

ObjectId('5a70a9093207864b03da16b2')

In [94]:
gf = magresFilesFS.get(ObjectId('5a6f0c0c3207864b03da169a'))

NoFile: no file in gridfs collection Collection(Database(MongoClient(host=['wigner.esc.rl.ac.uk:27017'], document_class=dict, tz_aware=False, connect=True), u'ccpnc'), u'magresFilesFS.files') with _id ObjectId('5a6f0c0c3207864b03da169a')

In [92]:
print gf.read()

#$magres-abinitio-v1.0
# See URL for format definition and code samples and libraries
[calculation]
calc_code CASTEP
calc_code_version 6.1
calc_code_hgversion 54e4ee5bcada+ magres-format 2012-09-04                                          
calc_code_platform linux_x86_64_ifort12            
calc_name ethanol
calc_comment                                                                                 
calc_xcfunctional PBE            
calc_cutoffenergy          2.5000000000000000E+01
calc_kpoint_mp_grid           -1          -1          -1
calc_kpoint_mp_offset       0.00000000      0.00000000      0.00000000
[/calculation]
[atoms]
units lattice Angstrom
lattice    6.0000000000000009E+00   0.0000000000000000E+00   0.0000000000000000E+00   0.0000000000000000E+00   6.0000000000000009E+00   0.0000000000000000E+00   0.0000000000000000E+00   0.0000000000000000E+00   6.0000000000000009E+00
units atom Angstrom
atom H       H                  1          3.9805990000000007E+00          4.1783420

In [57]:
databaseSearch([{'type': 'orcid', 'args': {'orcid': '0000-0003-4851-1144'}}])

{u'chemname': u'thingNow', u'metadataID': u'5a6f0c0c3207864b03da1689', u'latest_version': {u'magresFilesID': u'5a6f0c0c3207864b03da168a'}, u'values': [{u'iso': [29.559937639131203, 30.22618664854139, 30.072254456072027, 26.953990818833876, 27.373946759128433, 31.888119371185127], u'species': u'H'}, {u'iso': [156.1229153499832, 109.35753044488325], u'species': u'C'}, {u'iso': [267.0122765992025], u'species': u'O'}], u'orcid': {u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}, u'formula': [{u'species': u'C', u'n': 2}, {u'species': u'H', u'n': 6}, {u'species': u'O', u'n': 1}], u'_id': ObjectId('5a6f0c0c3207864b03da168c')}
{u'chemname': u'thingNow', u'metadataID': u'5a6f0c853207864b03da1691', u'latest_version': {u'magresFilesID': u'5a6f0c853207864b03da1692'}, u'values': [{u'iso': [29.559937639131203, 30.22618664854139, 30.072254456072027, 26.953990818833876, 27.373946759128433, 31.888119371185127], u'species': u'H'}, {u'iso': [156.122

'[null, null, {"chemname": "thingNow", "doi": ""}]'

In [48]:
ccpnc.collection_names()

[u'magresFilesFS',
 u'magresFilesFS.chunks',
 u'magresData',
 u'magresIndex',
 u'magresFilesFS.files',
 u'magresFiles',
 u'magresMetadata']

In [81]:
mfile = open('tests/ethanol.magres').read()
addMagresFile(mfile, chemname='thingNow', orcid={u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'})

1

In [25]:
def findById(fileID):
    return ccpnc.magresFiles.find({'_id': ObjectId(fileID)})

In [75]:
ff = ccpnc.magresIndex.find({"chemname": "thingNow"})
# Get the first magres file in the query
mfiles = []
for f in ff:
    print f['_id']
    print ccpnc.magresMetadata.find({'_id': ObjectId(f['metadataID'])}).next()

In [76]:
for f in ccpnc.magresIndex.find({'_id': ObjectId('5a6f0c0c3207864b03da168c')}):
    print f

In [79]:
removeMagresFiles('5a6f0caf3207864b03da169a')

RuntimeError: No entry with that id found

In [38]:
ff = ccpnc.magresMetadata.find({"chemname": "thingNow"})

for f in ff:
    print f['version_history']

[{u'magresFilesID': ObjectId('5a6b61930c81be53f30a4470')}]


In [19]:
ff = ccpnc.magresData.find({"orcid.path": "0000-0003-4851-1144"})
for f in ff:
    print f['orcid']

{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}


In [6]:
from db_schema import magresDataSchema

In [12]:
d = f.copy()
del d['_id']
del d['magresFilesID']

d['csd-ref'] = {
    'refcode': 'C6H6OH',
    'digits': '01'
}

magresDataSchema.validate(d)

{u'chemname': u'Ethanol (additional file)',
 'csd-ref': {'digits': '01', 'refcode': 'C6H6OH'},
 u'doi': u'10.0001',
 u'orcid': {u'host': u'orcid.org',
  u'path': u'0000-0003-4851-1144',
  u'uri': u'http://orcid.org/0000-0003-4851-1144'},
 u'values': [{u'iso': [29.559937639131203,
    30.22618664854139,
    30.072254456072027,
    26.953990818833876,
    27.373946759128433,
    31.888119371185127],
   u'species': u'H'},
  {u'iso': [156.1229153499832, 109.35753044488325], u'species': u'C'},
  {u'iso': [267.0122765992025], u'species': u'O'}]}

In [8]:
import re

In [9]:
myre = re.compile('[0-9]{2}\Z')

In [10]:
myre.match('34')